In [16]:
import pandas as pd
import numpy as np
import os
from sklearn.neighbors import radius_neighbors_graph

In [39]:
train_dir = os.path.join("data", "raw", "Records", "train_po")
bg_dir = os.path.join("data", "raw", "Records", "train_bg")

regions = ["AWT", "CAN", "NSW", "NZ", "SA", "SWI"]

region = "AWT"

train_df = pd.read_csv(os.path.join(train_dir, f"{region}train_po.csv"))
bg_df = pd.read_csv(os.path.join(bg_dir, f"{region}train_bg.csv"))

In [40]:
train_df[train_df[['x', 'y']].duplicated(keep=False)].sort_values(by=['x', 'y'])

,siteid,spid,x,y,occ,group,bc01,bc04,bc05,bc06,bc12,bc15,bc17,bc20,bc31,bc33,slope,topo,tri
876,3566,awt06,283936.0,8207073.0,1,bird,18.2,0.97,26.7,9.7,2215,84,157,18.6,28,0.48,6.30,23,182.4171
2099,2332,awt15,283936.0,8207073.0,1,bird,18.2,0.97,26.7,9.7,2215,84,157,18.6,28,0.48,6.30,23,182.4171
2353,6429,awt16,283936.0,8207073.0,1,bird,18.2,0.97,26.7,9.7,2215,84,157,18.6,28,0.48,6.30,23,182.4171
879,3569,awt06,286613.0,8196773.0,1,bird,18.0,0.97,26.7,9.5,2218,83,157,18.6,27,0.49,6.85,0,295.3015
2497,471,awt17,286613.0,8196773.0,1,bird,18.0,0.97,26.7,9.5,2218,83,157,18.6,27,0.49,6.85,0,295.3015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,6618,awt16,496846.0,7844604.0,1,bird,17.6,1.17,26.8,8.5,2133,88,155,18.4,20,0.62,13.60,95,1169.6800
1139,3887,awt06,498367.0,7847707.0,1,bird,22.5,1.06,30.5,12.4,1084,107,51,19.6,66,0.12,30.50,48,1007.5750
2922,5242,awt18,498367.0,7847707.0,1,bird,22.5,1.06,30.5,12.4,1084,107,51,19.6,66,0.12,30.50,48,1007.5750
1140,3888,awt06,499060.0,7847970.0,1,bird,23.1,1.05,30.9,12.8,1055,107,48,19.6,69,0.11,16.30,8,490.4906


In [41]:
spacial_cols = ['x', 'y']
species_cols = ['group']
env_cols = ['occ', 'bc01', 'bc04', 'bc05', 'bc06', 'bc12', 'bc15', 'bc17', 'bc20', 'bc31', 'bc33','slope','topo', 'tri']

location_cols = spacial_cols + env_cols

train_df['location_group'] = train_df.groupby(location_cols).ngroup()
train_df.sort_values(by=['location_group'])

,siteid,spid,x,y,occ,group,bc01,bc04,bc05,bc06,bc12,bc15,bc17,bc20,bc31,bc33,slope,topo,tri,location_group
3729,23748,awt39,282790.9375,8202508.5,1,plant,18.2,0.97,26.8,9.7,2143,84,148,18.6,29,0.46,9.292216,0,201.4919,0
2099,2332,awt15,283936.0000,8207073.0,1,bird,18.2,0.97,26.7,9.7,2215,84,157,18.6,28,0.48,6.300000,23,182.4171,1
2353,6429,awt16,283936.0000,8207073.0,1,bird,18.2,0.97,26.7,9.7,2215,84,157,18.6,28,0.48,6.300000,23,182.4171,1
876,3566,awt06,283936.0000,8207073.0,1,bird,18.2,0.97,26.7,9.7,2215,84,157,18.6,28,0.48,6.300000,23,182.4171,1
877,3567,awt06,285213.0000,8213129.0,1,bird,18.5,0.96,26.9,10.1,2173,84,152,18.6,29,0.45,6.600000,0,260.2191,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,6427,awt08,498282.0000,7848637.0,1,bird,23.2,1.04,31.0,13.0,1045,108,46,19.6,70,0.11,9.420000,3,489.5263,1169
2922,5242,awt18,498367.0000,7847707.0,1,bird,22.5,1.06,30.5,12.4,1084,107,51,19.6,66,0.12,30.500000,48,1007.5750,1170
1139,3887,awt06,498367.0000,7847707.0,1,bird,22.5,1.06,30.5,12.4,1084,107,51,19.6,66,0.12,30.500000,48,1007.5750,1170
1529,6428,awt08,499060.0000,7847970.0,1,bird,23.1,1.05,30.9,12.8,1055,107,48,19.6,69,0.11,16.300000,8,490.4906,1171


In [43]:
bg_df['location_group'] = bg_df.groupby(location_cols).ngroup()
bg_df.sort_values(by=['location_group'])

,siteid,spid,x,y,occ,group,bc01,bc04,bc05,bc06,bc12,bc15,bc17,bc20,bc31,bc33,slope,topo,tri,location_group
823,100824,pseudo1,255735.8252,8200887.848,0,NaN,21.6,0.95,30.0,11.600000,912,114,0,20.0,85,0.04,4.202670,100,116.193800,0
8251,108252,pseudo1,255975.8252,8203527.848,0,NaN,21.6,0.96,29.9,11.600000,925,113,0,20.0,84,0.04,0.721318,0,27.892650,1
7913,107914,pseudo1,256055.8252,8203207.848,0,NaN,21.6,0.96,29.9,11.600000,925,113,0,20.0,84,0.04,0.719450,5,31.080540,2
3204,103205,pseudo1,256135.8252,8197287.848,0,NaN,21.7,0.95,30.1,11.600000,897,114,0,20.0,86,0.04,13.398040,76,294.486000,3
9119,109120,pseudo1,256215.8252,8201367.848,0,NaN,21.6,0.96,30.0,11.600000,920,113,0,20.0,84,0.04,2.296861,50,65.863490,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,102137,pseudo1,499255.8252,7839527.848,0,NaN,19.9,1.13,28.8,10.100000,1374,98,84,19.2,45,0.23,30.260740,65,874.713100,9995
8150,108151,pseudo1,499255.8252,7847447.848,0,NaN,23.0,1.05,30.8,12.700000,1058,107,48,19.6,69,0.11,5.144219,31,259.034800,9996
4756,104757,pseudo1,499335.8252,7840407.848,0,NaN,19.7,1.13,28.6,9.900001,1448,97,90,19.1,42,0.27,32.018440,100,1030.272000,9997
6049,106050,pseudo1,499495.8252,7830087.848,0,NaN,23.6,1.09,31.9,12.200000,942,106,42,19.8,75,0.09,1.569068,77,80.249610,9998


In [44]:
bg_df['location_group'].nunique()

10000

In [18]:
spatial_graph = radius_neighbors_graph(
    train_df[['x', 'y']].values,
    radius=7500, # latitude and logitude are measured in meters? 
    mode='connectivity'
)

row_indices, col_indices = spatial_graph.nonzero()



(array([   0,    0,    0, ..., 3805, 3805, 3805], dtype=int32),
 array([ 885, 2358, 3003, ..., 3747, 3748, 3358], dtype=int32))

In [15]:
print(spatial_graph[0])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 72 stored elements and shape (1, 3806)>
  Coords	Values
  (0, 885)	1.0
  (0, 2358)	1.0
  (0, 3003)	1.0
  (0, 763)	1.0
  (0, 1644)	1.0
  (0, 2001)	1.0
  (0, 3364)	1.0
  (0, 3731)	1.0
  (0, 1311)	1.0
  (0, 888)	1.0
  (0, 340)	1.0
  (0, 181)	1.0
  (0, 3006)	1.0
  (0, 1143)	1.0
  (0, 1765)	1.0
  (0, 183)	1.0
  (0, 3246)	1.0
  (0, 2684)	1.0
  (0, 3689)	1.0
  (0, 185)	1.0
  (0, 564)	1.0
  (0, 7)	1.0
  (0, 2687)	1.0
  (0, 3662)	1.0
  (0, 1302)	1.0
  :	:
  (0, 3661)	1.0
  (0, 3361)	1.0
  (0, 3122)	1.0
  (0, 3604)	1.0
  (0, 1308)	1.0
  (0, 3605)	1.0
  (0, 3245)	1.0
  (0, 3730)	1.0
  (0, 3134)	1.0
  (0, 335)	1.0
  (0, 1301)	1.0
  (0, 3440)	1.0
  (0, 3002)	1.0
  (0, 180)	1.0
  (0, 3660)	1.0
  (0, 3560)	1.0
  (0, 3362)	1.0
  (0, 3603)	1.0
  (0, 1142)	1.0
  (0, 3441)	1.0
  (0, 3244)	1.0
  (0, 1300)	1.0
  (0, 179)	1.0
  (0, 902)	1.0
  (0, 893)	1.0
